In [1]:
import pandas as pd
import numpy as np

In [ ]:
data_not_processed = pd.read_csv('Features_24048_ordered.csv')
data_not_processed

In [ ]:
data_not_processed['labels']

## Посмотрим как обстоят дела с обучением на неотфильтрованной и немасштабированной выборке

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

In [ ]:
param_grid = {
    'alpha' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter' : [1000],
}

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
mlp = MLPRegressor()

In [ ]:
X = data_not_processed.drop(['labels'] ,axis=1)
y = data_not_processed['labels']

In [ ]:
grid = GridSearchCV(mlp, param_grid, verbose=4, cv=2)

In [ ]:
grid.fit(X, y)

In [ ]:
grid.best_params_["alpha"]

In [ ]:
X_test = data_not_processed.loc[(data_not_processed['film'] == 41)].reset_index(drop=True)
X_train = data_not_processed.loc[(data_not_processed['film'] != 41)].reset_index(drop=True)
X_train = X_train.sample(frac=1)
X_test = X_test.sample(frac=1)

y_train = X_train['labels']
y_test = X_test['labels']
train_film = X_train['film']
train_subj = X_train['Subj']
train_ch = X_train['ch']
test_film = X_test['film']
test_subj = X_test['Subj']
test_ch = np.array(X_test['ch'])

X_train = X_train.drop(['film', 'ch', 'Subj', 'labels'], axis=1)
X_test = X_test.drop(['film', 'ch', 'Subj', 'labels'], axis=1)

In [ ]:
preds = MLPRegressor(alpha=1000, max_iter=1000).fit(X_train, y_train).predict(X_test)

In [ ]:
chans = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
pred = [[0 for j in range(8)] for i in range(42)]
num_chans = [[0 for j in range(8)] for i in range(42)]
labels = [[0 for j in range(8)] for i in range(42)]
lbl = [[0 for j in range(8)] for i in range(42)]
channels = [[] for i in range(40)]
for i in range(len(preds)):
    pred[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += np.array(preds)[i]
    num_chans[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += 1
    labels[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += np.array(y_test)[i]
    channels[np.array(test_subj)[i]].append(chans[test_ch[i]])
labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
pred = np.array(pred).astype(np.float64) / np.array(num_chans).astype(np.float64)
labels = np.nan_to_num(labels)
pred = np.nan_to_num(pred)
predicted = []
lbl = []
for i in range(len(labels)):
    if sum(labels[i]) != 0:
        for j in range(8):
            if labels[i][j] != 0:
                lbl.append(labels[i][j])  
                predicted.append(pred[i][j])

In [ ]:
for [l, p] in zip(lbl, predicted):
    print('label : {:0.2f} predicted: {:0.2f} '.format(l, p))

## Все плохо 

Откроем данные с категориальными признаками

In [39]:
data_not_processed = pd.read_csv('Features_no_filtering_ordered.csv')
data_not_processed

,teager_kaiser_energy_6mean,time_corr_F8,max_cross_corr_C4,fractal_katz,phase_lock_val_F3,fisher_information,decorr_time,max_cross_corr_T8,max_cross_corr_P3,spect_corr_C4,...,Subj_34,Subj_35,Subj_36,Subj_37,Subj_38,Subj_39,labels,film,ch,Subj
0,1.116430e-04,0.218165,0.516112,0.363249,0.549476,0.279508,0.744318,0.788015,0.427071,0.295006,...,0.0,0.0,0.0,0.0,0.0,0.0,7.83,1,0,1
1,1.222589e-04,0.294963,0.412961,0.302930,0.000000,0.284011,0.750000,0.674391,0.210305,0.406826,...,0.0,0.0,0.0,0.0,0.0,0.0,7.83,1,1,1
2,7.987486e-05,0.256055,0.543653,0.298399,0.351609,0.274894,0.750000,0.660297,0.079877,0.553034,...,0.0,0.0,0.0,0.0,0.0,0.0,7.83,1,2,1
3,2.984681e-04,0.346090,0.389749,0.362881,0.495271,0.198436,0.750000,0.627896,0.152767,0.794022,...,0.0,0.0,0.0,0.0,0.0,0.0,7.83,1,3,1
4,1.145732e-04,0.584050,0.108225,0.294442,0.316123,0.309459,0.750000,0.518746,0.556116,0.409080,...,0.0,0.0,0.0,0.0,0.0,0.0,7.83,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24043,4.222781e-10,0.449646,0.778307,0.344409,0.102099,0.547843,0.000000,0.400588,0.684508,0.809331,...,0.0,0.0,0.0,0.0,0.0,1.0,9.00,41,13,39
24044,3.903128e-10,0.734446,0.000000,0.350095,0.079309,0.470511,0.000000,0.701760,0.535743,0.499011,...,0.0,0.0,0.0,0.0,0.0,1.0,9.00,41,14,39
24045,1.437144e-09,0.838142,0.700925,0.440958,0.058883,0.283860,0.000000,0.000000,0.186631,0.140640,...,0.0,0.0,0.0,0.0,0.0,1.0,9.00,41,15,39
24046,5.283381e-10,0.836981,0.838758,0.361852,0.079258,0.480124,0.000000,0.755741,0.309949,0.800837,...,0.0,0.0,0.0,0.0,0.0,1.0,9.00,41,16,39


In [4]:
param_grid = {
    'alpha' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter' : [1000],
}

In [5]:
mlp = MLPRegressor()

In [6]:
X = data_not_processed.drop(['labels'] ,axis=1)
y = data_not_processed['labels']

In [7]:
grid = GridSearchCV(mlp, param_grid, verbose=4, cv=2)

In [8]:
grid.fit(X, y)

Fitting 2 folds for each of 7 candidates, totalling 14 fits
[CV 1/2] END .......alpha=0.001, max_iter=1000;, score=-0.165 total time=  37.2s
[CV 2/2] END .......alpha=0.001, max_iter=1000;, score=-0.466 total time=  35.4s
[CV 1/2] END ........alpha=0.01, max_iter=1000;, score=-2.281 total time=  49.7s
[CV 2/2] END ........alpha=0.01, max_iter=1000;, score=-1.686 total time=  45.2s
[CV 1/2] END .........alpha=0.1, max_iter=1000;, score=-0.614 total time=  47.0s
[CV 2/2] END .........alpha=0.1, max_iter=1000;, score=-0.563 total time=  39.5s
[CV 1/2] END ...........alpha=1, max_iter=1000;, score=-0.208 total time=  45.0s
[CV 2/2] END ...........alpha=1, max_iter=1000;, score=-0.145 total time=  43.3s
[CV 1/2] END ..........alpha=10, max_iter=1000;, score=-0.018 total time=  15.2s
[CV 2/2] END ...........alpha=10, max_iter=1000;, score=0.184 total time=  13.3s
[CV 1/2] END .........alpha=100, max_iter=1000;, score=-0.259 total time=  29.4s
[CV 2/2] END .........alpha=100, max_iter=1000;, 

GridSearchCV(cv=2, estimator=MLPRegressor(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'max_iter': [1000]},
             verbose=4)

In [9]:
grid.best_params_["alpha"]

1000

In [68]:
X_test = data_not_processed.loc[(data_not_processed['film'] == 41)].reset_index(drop=True)
X_train = data_not_processed.loc[(data_not_processed['film'] != 41)].reset_index(drop=True)
X_train = X_train.sample(frac=1)
X_test = X_test.sample(frac=1)

y_train = X_train['labels']
y_test = X_test['labels']
train_film = X_train['film']
train_subj = X_train['Subj']
train_ch = X_train['ch']
test_film = X_test['film']
test_subj = X_test['Subj']
test_ch = np.array(X_test['ch'])

X_train = X_train.drop(['film', 'ch', 'Subj', 'labels'], axis=1)
X_test = X_test.drop(['film', 'ch', 'Subj', 'labels'], axis=1)

In [11]:
preds = MLPRegressor(alpha=1000, max_iter=1000).fit(X_train, y_train).predict(X_test)

In [14]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error

In [16]:
print('r2:', r2_score(y_test, preds))
print('MAPE:', mean_absolute_percentage_error(y_test, preds))
print('MAE : ', mean_absolute_error(y_test, preds))

r2: -0.8538315527693034
MAPE: 0.3946519744544896
MAE :  2.8381278461216923


In [17]:
chans = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
pred = [[0 for j in range(8)] for i in range(42)]
num_chans = [[0 for j in range(8)] for i in range(42)]
labels = [[0 for j in range(8)] for i in range(42)]
lbl = [[0 for j in range(8)] for i in range(42)]
channels = [[] for i in range(40)]
for i in range(len(preds)):
    pred[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += np.array(preds)[i]
    num_chans[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += 1
    labels[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += np.array(y_test)[i]
    channels[np.array(test_subj)[i]].append(chans[test_ch[i]])
labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
pred = np.array(pred).astype(np.float64) / np.array(num_chans).astype(np.float64)
labels = np.nan_to_num(labels)
pred = np.nan_to_num(pred)
predicted = []
lbl = []
for i in range(len(labels)):
    if sum(labels[i]) != 0:
        for j in range(8):
            if labels[i][j] != 0:
                lbl.append(labels[i][j])  
                predicted.append(pred[i][j])

<ipython-input-17-373309886a93>:12: RuntimeWarning: invalid value encountered in true_divide
  labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
<ipython-input-17-373309886a93>:13: RuntimeWarning: invalid value encountered in true_divide
  pred = np.array(pred).astype(np.float64) / np.array(num_chans).astype(np.float64)


In [18]:
for [l, p] in zip(lbl, predicted):
    print('label : {:0.2f} predicted: {:0.2f} '.format(l, p))

label : 10.00 predicted: 5.43 
label : 10.00 predicted: 5.43 
label : 6.00 predicted: 5.43 
label : 8.00 predicted: 5.43 
label : 3.00 predicted: 5.43 
label : 7.00 predicted: 5.43 
label : 9.00 predicted: 5.43 


In [19]:
print('r2:', r2_score(lbl, predicted))
print('MAPE:', mean_absolute_percentage_error(lbl, predicted))
print('MAE:', mean_absolute_error(lbl ,predicted))

r2: -0.8538315522682742
MAPE: 0.3946519744544896
MAE: 2.8381278461216928


## Попробуем применить QuantileTransformer

In [59]:
from sklearn.preprocessing import QuantileTransformer

In [69]:
qt_train = QuantileTransformer(output_distribution='normal',
                         copy=True)
y_train = qt_train.fit_transform(np.array(y_train).reshape(-1, 1))
qt_test = QuantileTransformer(output_distribution='normal',
                         copy=True)
y_test = qt_test.fit_transform(np.array(y_test).reshape(-1, 1))

C:\Users\po6wi\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2590: UserWarning: n_quantiles (1000) is greater than the total number of samples (126). n_quantiles is set to n_samples.
  warnings.warn(


In [70]:
model = MLPRegressor(random_state = 1, max_iter=1000).fit(X_train, y_train.reshape(-1))
y_pred = model.predict(X_test)

In [71]:
preds = qt_test.inverse_transform(preds.reshape(-1, 1))

In [72]:
y_test = qt_test.inverse_transform(np.array(y_test).reshape(-1, 1))

In [ ]:
preds

In [35]:
chans = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
pred = [[0 for j in range(8)] for i in range(42)]
num_chans = [[0 for j in range(8)] for i in range(42)]
labels = [[0 for j in range(8)] for i in range(42)]
lbl = [[0 for j in range(8)] for i in range(42)]
channels = [[] for i in range(40)]
for i in range(len(preds)):
    pred[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += np.array(preds)[i]
    num_chans[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += 1
    labels[np.array(test_subj)[i]][np.array(test_film)[i] - 41] += np.array(y_test)[i]
    channels[np.array(test_subj)[i]].append(chans[test_ch[i]])
labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
pred = np.array(pred).astype(np.float64) / np.array(num_chans).astype(np.float64)
labels = np.nan_to_num(labels)
pred = np.nan_to_num(pred)
predicted = []
lbl = []
for i in range(len(labels)):
    if sum(labels[i]) != 0:
        for j in range(8):
            if labels[i][j] != 0:
                lbl.append(labels[i][j])  
                predicted.append(pred[i][j])

<ipython-input-35-373309886a93>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
<ipython-input-35-373309886a93>:12: RuntimeWarning: invalid value encountered in true_divide
  labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
<ipython-input-35-373309886a93>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pred = np.array(pred).astype(np.float64) / np.array(num_chans).astype(np.float64)
<ipython-input-35-373309886a93>:13: Runtime

In [36]:
for [l, p] in zip(lbl, predicted):
    print('label : {:0.2f} predicted: {:0.2f} '.format(l, p))

label : 10.00 predicted: 7.64 
label : 10.00 predicted: 7.64 
label : 6.00 predicted: 7.64 
label : 8.00 predicted: 7.64 
label : 3.00 predicted: 7.64 
label : 7.00 predicted: 7.64 
label : 9.00 predicted: 7.64 
